In [28]:
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
Blinks = np.load('/content/drive/MyDrive/DLA/npy/Blinks_F3.npy')
Labels = np.load('/content/drive/MyDrive/DLA/npy/Labels_F3.npy')
BlinksTest = np.load('/content/drive/MyDrive/DLA/npy/BlinksTest_F3.npy')
LabelsTest = np.load('/content/drive/MyDrive/DLA/npy/LabelsTest_F3.npy')
    #deciding the indices of each video based on the fold
    #####################Normalizing the input#############Second phase
BlinksTest[:,:,0]=(BlinksTest[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
Blinks[:,:,0]=(Blinks[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
    #####
    #####
BlinksTest[:,:,1]=(BlinksTest[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
Blinks[:,:,1]=(Blinks[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
    #####
BlinksTest[:,:,2]=(BlinksTest[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
Blinks[:,:,2]=(Blinks[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
    #####
BlinksTest[:,:,3]=(BlinksTest[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
Blinks[:,:,3]=(Blinks[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
    ####
    ####JUST TO DOUBLE CHECK
    ####
    # print(np.mean(Blinks[:,:,0]),np.mean(Blinks[:,:,1]),np.mean(Blinks[:,:,2]),np.mean(Blinks[:,:,3]))
    # print(np.std(Blinks[:,:,0]),np.std(Blinks[:,:,1]),np.std(Blinks[:,:,2]),np.std(Blinks[:,:,3]))
    # print(np.mean(BlinksTest[:,:,0]),np.mean(BlinksTest[:,:,1]),np.mean(BlinksTest[:,:,2]),np.mean(BlinksTest[:,:,3]))
    # print(np.std(BlinksTest[:,:,0]),np.std(BlinksTest[:,:,1]),np.std(BlinksTest[:,:,2]),np.std(BlinksTest[:,:,3]))

In [48]:
#BinarizationLabels
LabelsTest[LabelsTest==10] = 1
Labels[Labels==10] = 1

In [82]:
def create_model(model_name):
    if model_name == 'LSTMreza':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(64, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.1)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'LSTM':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Masking(mask_value=0.0)(input)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.5)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'stackedreza':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(64, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      rnn_cells = [tf.keras.layers.LSTMCell(32, dropout=0.1, recurrent_dropout=0.1) for _ in range(4)]
      stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)
      x = tf.keras.layers.RNN(stacked_lstm)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.1)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    else:
      print("Chiama le cose con il loro nome")
    return model

In [83]:
model_name = "stackedreza"
model = create_model(model_name)

In [84]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 30, 4)]           0         
_________________________________________________________________
dense_50 (Dense)             (None, 30, 64)            320       
_________________________________________________________________
masking_17 (Masking)         (None, 30, 64)            0         
_________________________________________________________________
rnn_11 (RNN)                 (None, 32)                37376     
_________________________________________________________________
dense_51 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_16 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 16)                272 

In [87]:
lr = 0.000053
bs = 64
ep = 80
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy']
)

In [ ]:
model.fit(x= Blinks, y = Labels, validation_data=(BlinksTest, LabelsTest), epochs=ep, batch_size=bs)

Epoch 1/80
121/121 [==============================] - 30s 163ms/step - loss: 1.7782 - accuracy: 0.4474 - val_loss: 1.6688 - val_accuracy: 0.2994
Epoch 2/80
121/121 [==============================] - 19s 161ms/step - loss: 1.0469 - accuracy: 0.4672 - val_loss: 0.7380 - val_accuracy: 0.4038
Epoch 3/80
121/121 [==============================] - 19s 160ms/step - loss: 0.7752 - accuracy: 0.5442 - val_loss: 0.6584 - val_accuracy: 0.6775
Epoch 4/80
121/121 [==============================] - 19s 158ms/step - loss: 0.7219 - accuracy: 0.5469 - val_loss: 0.6245 - val_accuracy: 0.7645
Epoch 5/80
121/121 [==============================] - 19s 157ms/step - loss: 0.7348 - accuracy: 0.5383 - val_loss: 0.5938 - val_accuracy: 0.7379
Epoch 6/80
121/121 [==============================] - 19s 158ms/step - loss: 0.7080 - accuracy: 0.5481 - val_loss: 0.5721 - val_accuracy: 0.7819
Epoch 7/80
121/121 [==============================] - 19s 157ms/step - loss: 0.7186 - accuracy: 0.5594 - val_loss: 0.5432 - val_ac

In [ ]:
tf.compat.v1.disable_eager_execution()
training = tf.compat.v1.placeholder(tf.bool,name='phase_train')